In [17]:
import pandas as pd
from IPython.display import clear_output
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read the data
data_import = pd.read_csv('TSLA.csv', parse_dates=["Date"])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500
1,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500
2,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000
3,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000
4,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500
...,...,...,...,...,...,...,...
3035,2022-07-20,740.349976,751.989990,730.450012,742.500000,742.500000,29621400
3036,2022-07-21,765.320007,819.799988,764.599976,815.119995,815.119995,47344100
3037,2022-07-22,828.659973,842.359985,812.140015,816.729980,816.729980,34421200
3038,2022-07-25,816.650024,822.440002,802.200012,805.299988,805.299988,21357800


In [3]:
# Add a column of the day names

day_mapper = {0: "Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
data_import["DayOfWeek"] = data_import["Date"].map(lambda x: day_mapper[x.dayofweek])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
0,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500,Tuesday
1,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500,Wednesday
2,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000,Thursday
3,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000,Friday
4,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500,Tuesday
...,...,...,...,...,...,...,...,...
3035,2022-07-20,740.349976,751.989990,730.450012,742.500000,742.500000,29621400,Wednesday
3036,2022-07-21,765.320007,819.799988,764.599976,815.119995,815.119995,47344100,Thursday
3037,2022-07-22,828.659973,842.359985,812.140015,816.729980,816.729980,34421200,Friday
3038,2022-07-25,816.650024,822.440002,802.200012,805.299988,805.299988,21357800,Monday


# Method 1: Average Dip per Day

In [4]:
# Take the previous value and then subtract it from the current value, check the difference

data_import["diff_from_previous_day"] = (data_import["Open"].diff() / data_import["Open"]) * 100
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500,Tuesday,NaN
1,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500,Wednesday,26.328034
2,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000,Thursday,-3.160000
3,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000,Friday,-8.695652
4,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500,Tuesday,-15.000000
...,...,...,...,...,...,...,...,...,...
3035,2022-07-20,740.349976,751.989990,730.450012,742.500000,742.500000,29621400,Wednesday,0.722628
3036,2022-07-21,765.320007,819.799988,764.599976,815.119995,815.119995,47344100,Thursday,3.262692
3037,2022-07-22,828.659973,842.359985,812.140015,816.729980,816.729980,34421200,Friday,7.643662
3038,2022-07-25,816.650024,822.440002,802.200012,805.299988,805.299988,21357800,Monday,-1.470636


In [5]:
# Compare the different days to see which one has the biggest dip from the previous day

data_import.groupby("DayOfWeek")["diff_from_previous_day"].mean()

DayOfWeek
Friday      -0.069823
Monday       0.029360
Thursday     0.069844
Tuesday      0.417852
Wednesday    0.079324
Name: diff_from_previous_day, dtype: float64

 We see that the highest dip is on Monday. So, using this method it can be conluded that Monday is the best day to purchase TSLA stocks as it has the highest dip on average.

# Method 2: Backtester Method

Tracks the number of shares that I own based on the number of shares that I bought when and what happened to the price on any given day

In [8]:
#Create a table of every day between the min and max dates to deal with holidays
dates = pd.date_range(start=data_import['Date'].min(), end=data_import['Date'].max())

#Add the day of the week
date_table = pd.DataFrame(data={'Calendar Date':dates})
date_table["DayOfWeek"] = date_table["Calendar Date"].map(lambda x: day_mapper[x.dayofweek])
date_table

,Calendar Date,DayOfWeek
0,2010-06-29,Tuesday
1,2010-06-30,Wednesday
2,2010-07-01,Thursday
3,2010-07-02,Friday
4,2010-07-03,Saturday
...,...,...
4406,2022-07-22,Friday
4407,2022-07-23,Saturday
4408,2022-07-24,Sunday
4409,2022-07-25,Monday


In [11]:
# Create a table of all dates and remove weekends

full_calendar = pd.merge(left=date_table, right=data_import, how='left', left_on='Calendar Date', right_on='Date')
full_calendar = full_calendar[~full_calendar['DayOfWeek_x'].isin(['Saturday', 'Sunday'])]
full_calendar.rename(columns={'Date':'Trading Date',}, inplace=True)
full_calendar

,Calendar Date,DayOfWeek_x,Trading Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek_y,diff_from_previous_day
0,2010-06-29,Tuesday,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500.0,Tuesday,NaN
1,2010-06-30,Wednesday,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500.0,Wednesday,26.328034
2,2010-07-01,Thursday,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000.0,Thursday,-3.160000
3,2010-07-02,Friday,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000.0,Friday,-8.695652
6,2010-07-05,Monday,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4404,2022-07-20,Wednesday,2022-07-20,740.349976,751.989990,730.450012,742.500000,742.500000,29621400.0,Wednesday,0.722628
4405,2022-07-21,Thursday,2022-07-21,765.320007,819.799988,764.599976,815.119995,815.119995,47344100.0,Thursday,3.262692
4406,2022-07-22,Friday,2022-07-22,828.659973,842.359985,812.140015,816.729980,816.729980,34421200.0,Friday,7.643662
4409,2022-07-25,Monday,2022-07-25,816.650024,822.440002,802.200012,805.299988,805.299988,21357800.0,Monday,-1.470636


In [12]:
# Making the table more readable

full_calendar.rename(columns={'DayOfWeek_x':'DayOfWeek',}, inplace=True)
full_calendar.drop(columns='DayOfWeek_y', inplace=True)
full_calendar

,Calendar Date,DayOfWeek,Trading Date,Open,High,Low,Close,Adj Close,Volume,diff_from_previous_day
0,2010-06-29,Tuesday,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500.0,NaN
1,2010-06-30,Wednesday,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500.0,26.328034
2,2010-07-01,Thursday,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000.0,-3.160000
3,2010-07-02,Friday,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000.0,-8.695652
6,2010-07-05,Monday,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4404,2022-07-20,Wednesday,2022-07-20,740.349976,751.989990,730.450012,742.500000,742.500000,29621400.0,0.722628
4405,2022-07-21,Thursday,2022-07-21,765.320007,819.799988,764.599976,815.119995,815.119995,47344100.0,3.262692
4406,2022-07-22,Friday,2022-07-22,828.659973,842.359985,812.140015,816.729980,816.729980,34421200.0,7.643662
4409,2022-07-25,Monday,2022-07-25,816.650024,822.440002,802.200012,805.299988,805.299988,21357800.0,-1.470636


In [13]:
# To account for holidays, take the next days stock value

full_calendar = full_calendar.bfill(axis='rows').reset_index(drop=True)
full_calendar

,Calendar Date,DayOfWeek,Trading Date,Open,High,Low,Close,Adj Close,Volume,diff_from_previous_day
0,2010-06-29,Tuesday,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500.0,26.328034
1,2010-06-30,Wednesday,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500.0,26.328034
2,2010-07-01,Thursday,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000.0,-3.160000
3,2010-07-02,Friday,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000.0,-8.695652
4,2010-07-05,Monday,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500.0,-15.000000
...,...,...,...,...,...,...,...,...,...,...
3146,2022-07-20,Wednesday,2022-07-20,740.349976,751.989990,730.450012,742.500000,742.500000,29621400.0,0.722628
3147,2022-07-21,Thursday,2022-07-21,765.320007,819.799988,764.599976,815.119995,815.119995,47344100.0,3.262692
3148,2022-07-22,Friday,2022-07-22,828.659973,842.359985,812.140015,816.729980,816.729980,34421200.0,7.643662
3149,2022-07-25,Monday,2022-07-25,816.650024,822.440002,802.200012,805.299988,805.299988,21357800.0,-1.470636


In [14]:
#Checking if the method is accurate

full_calendar['DayOfWeek'].value_counts()

Tuesday      631
Wednesday    630
Thursday     630
Friday       630
Monday       630
Name: DayOfWeek, dtype: int64

In [15]:
# We create a function to backtest as if we invested every day from Monday to Friday from the beginning to see which day yields highest balance

def day_backtester(day, amount_to_invest, data):
    temp_data = data[data['DayOfWeek'] == day]
    temp_data["Shares Owned"] = amount_to_invest/temp_data['Open']
    final_price = temp_data['Open'].iloc[-1]
    final_amount = temp_data['Shares Owned'].sum() * final_price
    formatted_final_amount = "${:,.2f}".format(final_amount)
    return formatted_final_amount

In [18]:
for i in full_calendar['DayOfWeek'].unique():
    print(i, day_backtester(i, 500, full_calendar))

Tuesday $13,829,194.78
Wednesday $12,768,151.53
Thursday $13,140,938.93
Friday $14,232,337.35
Monday $14,020,397.12


This method shows us that investing everyday on Friday yields the highest account balance which contradicts the weekend effect, which states that Monday is usually the best day to purchase stocks as it's always the lowest on Mondays after the weekend.